In [615]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer

In [616]:
df = pd.read_csv('heart_disease_uci.csv')

In [617]:
df['trestbps'].describe()

count    861.000000
mean     132.132404
std       19.066070
min        0.000000
25%      120.000000
50%      130.000000
75%      140.000000
max      200.000000
Name: trestbps, dtype: float64

In [618]:
df

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,916,54,Female,VA Long Beach,asymptomatic,127.0,333.0,True,st-t abnormality,154.0,False,0.0,NaN,NaN,NaN,1
916,917,62,Male,VA Long Beach,typical angina,NaN,139.0,False,st-t abnormality,NaN,NaN,NaN,NaN,NaN,NaN,0
917,918,55,Male,VA Long Beach,asymptomatic,122.0,223.0,True,st-t abnormality,100.0,False,0.0,NaN,NaN,fixed defect,2
918,919,58,Male,VA Long Beach,asymptomatic,NaN,385.0,True,lv hypertrophy,NaN,NaN,NaN,NaN,NaN,NaN,0


In [619]:
df['chol'].replace(0, df['chol'].median(), inplace=True)
df['chol'].fillna(df['chol'].mean(), inplace=True)

#Drop all null values in restecg column
df.dropna(subset=['restecg'], inplace=True)

df['thalch'].fillna(df['thalch'].mean(), inplace=True)
df['trestbps'].fillna(df['trestbps'].mean(), inplace=True)
df.drop('id', inplace=True, axis=1)
df['exang'] = df['exang'].replace({False: 0, True: 1})
df['oldpeak'].fillna(df['oldpeak'].median(), inplace=True)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df['oldpeak'] = scaler.fit_transform(df[['oldpeak']])
#Apply label encoding to sex column

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['sex'] = le.fit_transform(df['sex'])

df.drop('ca', inplace=True, axis=1)
df.drop('fbs', inplace=True, axis=1)

df.dropna(subset=['exang'], inplace=True)

le = LabelEncoder()
df['slope'] = le.fit_transform(df['slope'])
df['slope'].value_counts()
df['slope'] = df['slope'].replace({3: np.nan})
imputer = KNNImputer(n_neighbors=10)
df['slope'] = imputer.fit_transform(df[['slope']])
df['slope'] = df['slope'].astype(int)

df = df[df['trestbps'] != 0]

df = pd.get_dummies(df, columns=['dataset'])
df = pd.get_dummies(df, columns=['restecg'])


cp_categories = ['typical angina', 'atypical angina', 'non-anginal', 'asymptomatic']
ordinal_encoder = OrdinalEncoder(categories=[cp_categories])
df['cp'] = ordinal_encoder.fit_transform(df[['cp']])

scaler = MinMaxScaler()
df['chol'] = scaler.fit_transform(df[['chol']])
df['trestbps'] = scaler.fit_transform(df[['trestbps']])

In [620]:
df.isnull().sum()

age                           0
sex                           0
cp                            0
trestbps                      0
chol                          0
thalch                        0
exang                         0
oldpeak                       0
slope                         0
thal                        430
num                           0
dataset_Cleveland             0
dataset_Hungary               0
dataset_Switzerland           0
dataset_VA Long Beach         0
restecg_lv hypertrophy        0
restecg_normal                0
restecg_st-t abnormality      0
dtype: int64

In [621]:
columns_to_convert = ['dataset_Cleveland', 'dataset_Hungary', 'dataset_Switzerland', 'dataset_VA Long Beach', 'restecg_lv hypertrophy', 'restecg_normal', 'restecg_st-t abnormality']
for col in columns_to_convert:
  df[col] = df[col].replace({True: 1, False: 0})

df


,age,sex,cp,trestbps,chol,thalch,exang,oldpeak,slope,thal,num,dataset_Cleveland,dataset_Hungary,dataset_Switzerland,dataset_VA Long Beach,restecg_lv hypertrophy,restecg_normal,restecg_st-t abnormality
0,63,1,0.0,0.541667,0.285714,150.0,0.0,0.556818,0,fixed defect,0,1,0,0,0,1,0,0
1,67,1,3.0,0.666667,0.388031,108.0,1.0,0.465909,1,normal,2,1,0,0,0,1,0,0
2,67,1,3.0,0.333333,0.277992,129.0,1.0,0.590909,1,reversable defect,1,1,0,0,0,1,0,0
3,37,1,2.0,0.416667,0.318533,187.0,0.0,0.693182,0,normal,0,1,0,0,0,0,1,0
4,41,0,1.0,0.416667,0.229730,172.0,0.0,0.454545,2,normal,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,62,1,3.0,0.650000,0.164093,138.0,1.0,0.295455,1,NaN,1,0,0,0,1,0,0,1
914,46,1,3.0,0.450000,0.434363,126.0,0.0,0.295455,1,normal,2,0,0,0,1,0,1,0
915,54,0,3.0,0.391667,0.478764,154.0,0.0,0.295455,1,NaN,1,0,0,0,1,0,0,1
917,55,1,3.0,0.350000,0.266409,100.0,0.0,0.295455,1,fixed defect,2,0,0,0,1,0,0,1


In [622]:
# prompt: Take the slope feature and run a classifier algorithm model on it to predict the null values and impute them



from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Split the data into training and testing sets
df_train = df.dropna()
df_test = df[df['thal'].isnull()]

# Prepare the features and target variable
X_train = df_train.drop('thal', axis=1)
y_train = df_train['thal']
X_test = df_test.drop('thal', axis=1)

# Train a RandomForestClassifier model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Predict the missing values
y_pred = model.predict(X_test)

# Impute the missing values in the original DataFrame
df.loc[df['thal'].isnull(), 'thal'] = y_pred

# Verify that there are no more missing values in the 'slope' column
print(df.isnull().sum())


age                         0
sex                         0
cp                          0
trestbps                    0
chol                        0
thalch                      0
exang                       0
oldpeak                     0
slope                       0
thal                        0
num                         0
dataset_Cleveland           0
dataset_Hungary             0
dataset_Switzerland         0
dataset_VA Long Beach       0
restecg_lv hypertrophy      0
restecg_normal              0
restecg_st-t abnormality    0
dtype: int64


In [625]:
# prompt: thal (Thalassemia): This column represents different types of thalassemia, which can be ordered by severity or risk.
#     3: normal
#     6: fixed defect
#     7: reversible defect
# Apply ordinal encoding to thal

thal_categories = ['normal', 'fixed defect', 'reversable defect']
ordinal_encoder = OrdinalEncoder(categories=[thal_categories])
df['thal'] = ordinal_encoder.fit_transform(df[['thal']])
df


,age,sex,cp,trestbps,chol,thalch,exang,oldpeak,slope,thal,num,dataset_Cleveland,dataset_Hungary,dataset_Switzerland,dataset_VA Long Beach,restecg_lv hypertrophy,restecg_normal,restecg_st-t abnormality
0,63,1,0.0,0.541667,0.285714,150.0,0.0,0.556818,0,1.0,0,1,0,0,0,1,0,0
1,67,1,3.0,0.666667,0.388031,108.0,1.0,0.465909,1,0.0,2,1,0,0,0,1,0,0
2,67,1,3.0,0.333333,0.277992,129.0,1.0,0.590909,1,2.0,1,1,0,0,0,1,0,0
3,37,1,2.0,0.416667,0.318533,187.0,0.0,0.693182,0,0.0,0,1,0,0,0,0,1,0
4,41,0,1.0,0.416667,0.229730,172.0,0.0,0.454545,2,0.0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,62,1,3.0,0.650000,0.164093,138.0,1.0,0.295455,1,2.0,1,0,0,0,1,0,0,1
914,46,1,3.0,0.450000,0.434363,126.0,0.0,0.295455,1,0.0,2,0,0,0,1,0,1,0
915,54,0,3.0,0.391667,0.478764,154.0,0.0,0.295455,1,2.0,1,0,0,0,1,0,0,1
917,55,1,3.0,0.350000,0.266409,100.0,0.0,0.295455,1,1.0,2,0,0,0,1,0,0,1


In [628]:
# prompt: Apply SVC model

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming 'target_variable' is the name of your target variable column
X = df.drop('num', axis=1)  # Replace 'target' with the actual column name
y = df['num']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an SVC model
model = SVC()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.43352601156069365


In [629]:
# prompt: Apply Logistic regression

from sklearn.linear_model import LogisticRegression

# Create a LogisticRegression model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.5433526011560693


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [630]:
# prompt: Apply random forest multi class classifier

# Create a RandomForestClassifier model
model = RandomForestClassifier()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.5491329479768786


In [631]:
# prompt: Apply xgboost

!pip install xgboost
from xgboost import XGBClassifier

# Create an XGBClassifier model
model = XGBClassifier()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.5780346820809249


In [632]:
# prompt: Apply lightgbm classifier

!pip install lightgbm
import lightgbm as lgb

# Create a LightGBM model
model = lgb.LGBMClassifier()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000141 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 346
[LightGBM] [Info] Number of data points in the train set: 689, number of used features: 17
[LightGBM] [Info] Start training from score -0.763800
[LightGBM] [Info] Start training from score -1.241936
[LightGBM] [Info] Start training from score -2.153215
[LightGBM] [Info] Start training from score -2.330549
[LightGBM] [Info] Start training from score -3.444199
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [633]:
# prompt: Apply decision trees

from sklearn.tree import DecisionTreeClassifier

# Create a DecisionTreeClassifier model
model = DecisionTreeClassifier()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.5606936416184971


In [634]:
# prompt: apply knn classifier

from sklearn.neighbors import KNeighborsClassifier

# Create a KNeighborsClassifier model
model = KNeighborsClassifier()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.4682080924855491
